In [3]:
# Realizamos los imports necesarios
import numpy as np
import re
import tensorflow as tf

## Definimos la red como ambiente

#### Clase que soporta las redes

In [17]:
class petri_net:
    
    def __init__(self,init_state,action_space,W_pre,W_post, name=None):
        self.name = None
        # Estados
        self.state = init_state
        self.init_state = init_state
        # Transiciones
        self.action_space = action_space
        self.T_mapped = {T:n for n,T in zip(range(len(self.action_space)), self.action_space)}
        # Matrices de incidencia
        self.W_pre = W_pre
        self.W_post= W_post
        self.W    = self.W_post - self.W_pre
        # Actualizamos la sensibilizacion por marcado
        self.T_shooteable_by_marked = [False for i in action_space]
        self.update_T_by_marks()
        
        
    def update_T_by_marks(self):
        for i in range(len(self.action_space)):
          action_idx = i
          marks_after_W_pre = self.state - self.W_pre[:,action_idx]
          self.T_shooteable_by_marked[action_idx] = False if any(n_tok < 0 for n_tok in marks_after_W_pre) else True

    def can_shoot(self,action):
        return self.T_shooteable_by_marked[action]

    def shoot_pn(self,action):
        self.state = self.state + self.W[:,action]
        self.update_T_by_marks()
        return self.state

    def reset(self):
        self.state = self.init_state
        self.update_T_by_marks()

        return tf.constant(self.state) 


#### Red Canal de Panama

In [18]:
def Panama():
  # Ejecutamos para canal de panamá
  name = "Canal_de_Panama"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11","T12","T13"]
  init_state   = np.array([3,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,3,1,1,3,1,3,1])
  R_vector     = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1]
  W_pre = np.array([
  [0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,1,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,1,0,0,0,0,0,0,1,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,1,0]])
      

  W_post = np.array([
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,1,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,1,0,0]
  ])

  return petri_net(init_state,action_space,W_pre,W_post, name=name)



#### MFC

In [19]:
# Ejecutamos para MFC
def MFC():
  name = "MFC"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11"]
  init_state   = np.array([0,0,0,0,0,0,0,0,0,0,1,2,1,2,1,3,3,1,1,2,2,2,2])
  R_vector     = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,1,1,1]
  W_pre = np.array([
  [0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,1],
  [1,0,0,0,0,0,0,0,0,0,1,0],
  [0,1,0,0,0,0,0,0,0,1,0,0],
  [0,0,1,0,0,0,0,0,1,0,0,0],
  [0,0,0,1,0,0,0,1,0,0,0,0],
  [0,0,0,0,1,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,1,0,0,0,0,0],
  [1,0,0,0,0,0,1,0,0,0,0,0],
  [1,0,0,0,0,0,1,0,0,0,0,0],
  [1,0,0,0,0,0,1,0,0,0,0,0]
  ])
      

  W_post = np.array([
  [1,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0],
  [0,1,0,0,0,0,0,0,0,0,0,1],
  [0,0,1,0,0,0,0,0,0,0,1,0],
  [0,0,0,1,0,0,0,0,0,1,0,0],
  [0,0,0,0,1,0,0,0,1,0,0,0],
  [0,0,0,0,0,1,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,1,0],
  [0,0,1,0,0,0,0,0,0,1,0,0],
  [0,0,0,1,0,0,0,0,1,0,0,0],
  [0,0,0,0,1,0,0,1,0,0,0,0]
  ])

  return petri_net(init_state,action_space,W_pre,W_post, name=name)

#### Red hospital

In [20]:
def Hospital():
  ### Ejecutar para red hospital
  name = "Hospital"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9"]
  init_state   = np.array([0,0,0,0,0,0,0,1,1,1,1,3,3,1,2,1])
  R_vector     = [1,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1]
  W_pre = np.array([
  [0,1,1,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,1],
  [1,0,0,0,0,0,0,0,1,0],
  [0,0,1,0,0,0,0,1,0,0],
  [0,0,0,1,1,0,1,0,0,0],
  [0,1,0,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0],
  [1,0,0,0,0,0,1,0,0,0],
  [1,0,0,0,0,0,1,0,0,0],
  [1,0,0,0,0,0,1,0,0,0]
  ])
      

  W_post = np.array([
  [1,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0],
  [0,0,0,1,1,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,1,0],
  [0,1,1,0,0,0,0,0,0,1],
  [0,0,0,0,1,0,0,0,1,0],
  [0,0,0,0,0,1,0,1,0,0],
  [0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1],
  [0,1,1,0,0,0,0,0,1,0],
  [0,0,0,1,1,0,0,0,1,0],
  [0,1,0,0,1,0,0,1,0,0]
  ])



  return petri_net(init_state,action_space,W_pre,W_post, name=name)

#### Red Guanjun

In [21]:
def Guanjun():
  ### Ejecutar para red guanjun
  name = "Guanjun"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11","T12","T13"]
  init_state   = np.array([0,0,0,0,0,0,0,0,0,0,1,1,3,3,3,3,3,1,6,5])
  R_vector     = [1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1]
  W_pre = np.array([
  [0,1,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,1,0,0,0,1,0],
  [0,0,0,0,1,0,1,0,0,1,0,1,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,1,0,0,1,0,0],
  [1,0,0,0,0,0,0,0,1,0,0,1,0,0]
  ])
      

  W_post = np.array([
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,1,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,1,0,0,0,1],
  [0,0,0,0,0,1,0,1,0,0,1,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,1,0,0,1,0,0,1,0],
  [0,1,0,0,1,0,0,0,0,1,0,0,1,0]
  ])

  return petri_net(init_state,action_space,W_pre,W_post, name=name)


#### Red Huang

In [22]:
def Huang():
  # Ejecutar para red Huang
  name = "Huang"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9"]
  init_state   = np.array([2,0,0,0,0,1,1,2,0,0,0,1,1,1])
  R_vector     = [0,1,0,0,0,0,0,0,0,0,0,0,1,0]
  W_pre = np.array([
  [1,0,0,0,0,0,0,0,0,0],
  [0,1,1,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,0,1,0,0],
  [1,0,0,1,1,0,1,0,1,0],
  [0,0,0,0,0,0,1,0,0,0]
  ])
      

  W_post = np.array([
  [0,0,0,0,0,1,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0],
  [0,0,0,1,1,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,1,0],
  [0,1,1,0,0,1,0,1,0,1],
  [0,0,0,0,0,0,0,0,1,0]
  ])

  return petri_net(init_state,action_space,W_pre,W_post, name=name)

#### Red Portugal

In [23]:
def Portugal():
  # Ejecutar para Red Portugal
  name = "Portugal"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11"]
  init_state   = np.array([3,0,0,3,0,0,3,0,0,3,0,0,2,2,2,1,1])
  R_vector     = [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1]
  W_pre = np.array([
  [1,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,1],
  [1,0,0,0,1,0,1,0,0,0,1,0],
  [1,0,0,1,0,0,1,0,0,1,0,0],
  [1,0,0,1,0,0,1,0,0,1,0,0],
  [0,1,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,1,0,0]
  ])
      

  W_post = np.array([
  [0,0,1,0,0,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0],
  [0,1,0,0,0,1,0,1,0,0,0,1],
  [0,1,0,0,1,0,0,1,0,0,1,0],
  [0,1,0,0,1,0,0,1,0,0,1,0],
  [0,0,1,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,1,0]
  ])

  return petri_net(init_state,action_space,W_pre,W_post, name=name)

#### Red POPN

In [14]:
def POPN():
  # Ejecutar para Red POPN
  name = "POPN"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11","T12","T13","T14","T15","T16","T17","T18","T19"]
  init_state   = np.array([1,0,0,0,0,0,1,1,1,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,1,1,1])
  R_idx = [6,7,8,9,13,15,21,26,27,28,29]
  R_vector     = [1 if i in R_idx else 0 for i in range(len(init_state))]

  W_pre = np.array([
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
  ])
      

  W_post = np.array([
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0],
  [0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0]
  ])

  return petri_net(init_state,action_space,W_pre,W_post, name=name)

#### Red Zhao


In [24]:
def Zhao():
  ### Ejecutar para red Zhao
  name = "Zhao"
  action_space = ["T0","T1","T2","T3","T4","T5","T6","T7","T8","T9","T10","T11","T12","T13"]
  init_state   = np.array([0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,3,1])
  R_vector     = [1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1]
  W_pre = np.array([
  [0,1,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,1,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [1,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,1,0,0,0,1,0,0,0,1,0,0,0,0],
  [0,0,1,0,0,0,1,0,1,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,1,0,0,0,0,0]
  ])
      

  W_post = np.array([
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,1,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,1,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,1,0,0,0,1,0,0,0,0,0,0],
  [1,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,1,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,1,0],
  [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,1,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,1,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,1,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,1],
  [0,0,0,0,0,0,0,0,0,0,0,1,0,0],
  [0,1,0,0,0,1,0,0,0,0,0,1,0,0],
  [0,0,1,0,0,0,1,0,0,0,1,0,0,0],
  [0,0,0,1,0,0,0,1,0,1,0,0,0,0],
  [0,1,0,0,0,1,0,0,0,0,1,0,0,0]
  ])

  return petri_net(init_state,action_space,W_pre,W_post, name=name)

## Testing

In [26]:
pn = Panama()
pn.init_state

array([3, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 1, 3, 1, 3,
       1])